In [1]:
import pandas as pd
import numpy as np
import mysql.connector 
import plotly.express as px
import json
from plotly.subplots import make_subplots
import plotly.graph_objects as go
class Data():
    def __init__(self,table_name,reporting_country=None,partner_code = None,year=None,period=None,section=None,SA_4=None,imp_exp=None):
        self.conn = mysql.connector.connect(user="root", password="123456",
                                       host="localhost",
                                       database="mexico_it",
                                       port='3306'
                                       )
        self.table_name = table_name
        self.reporting_country = reporting_country
        self.partner_code = partner_code
        self.year = year
        self.period = period
        self.section = section
        self.SA_4 = SA_4
        self.imp_exp = imp_exp



    def read_data(self):
        #diccionario con los parametros de la consulta
        diccionario_filtro = {'reporter_country':self.reporting_country,
                                'partner_code':self.partner_code,
                                'year':self.year,
                                'period':self.period,
                                'section':self.section,
                                'SA_4':self.SA_4,
                                'imp_exp':self.imp_exp}

        #parametros de la consulta que ingresó el usuario
        diccionario_filtro = {k: v for k, v in diccionario_filtro.items() if v is not None}
        line_ =''
        for k,v in diccionario_filtro.items():
            line_ = ''.join('{} in ({}) AND '.format(k, str(v).replace('[','').replace(']','')) for k, v in diccionario_filtro.items())[0:-4]
        
        #procesando table world_table
        query = "SELECT * FROM {} WHERE {}".format(self.table_name,line_)
        world_tradeTable = self.get_table(query)
        world_tradeTable['section'] = world_tradeTable['section'].astype(int)
        world_tradeTable = world_tradeTable[~(world_tradeTable['partner_code']=='0')] 
        world_tradeTable['porcentaje'] = world_tradeTable.groupby(['year','imp_exp','SA_4'],group_keys=False)['tradevalue'].apply(lambda x: (x/(x.sum()))*100)
        world_tradeTable['porcentaje'] = world_tradeTable['porcentaje'].round(2)
        world_tradeTable = world_tradeTable[world_tradeTable['porcentaje'] > 3] 

        #procesando table sa4_description
        query = "SELECT * FROM sections"
        sectionsTable = self.get_table(query)
        sectionsTable = sectionsTable.rename(columns={'id':'section'})

        #PROCESANDO TABLA COUNTRIES_
        query = "SELECT * FROM countries_"
        countriesTable_ = self.get_table(query)
        countriesTable_ = countriesTable_.rename(columns={'id':'partner_code'})
        
        #procesando table countries
        query = "SELECT * FROM countries"
        countriesTable = self.get_table(query)  
        countriesTable = countriesTable.rename(columns={'partner_code_':'partner_code'})
        countriesTable['partner_code'] = countriesTable['partner_code'].astype(str)
        df = world_tradeTable.merge(sectionsTable,how='left',on='section')
        df = df.merge(countriesTable,how='left',on='partner_code')
        df.drop(columns=['fobvalue','quantity_unit','netweight','reporter_country','tradequantity'],inplace=True)
        #dar formato a tradevalue en millones de dolares
        df['tradevalue'] = df['tradevalue'].astype(float)
        df['tradevalue'] = df['tradevalue'].apply(lambda x:x/1000000)
        #redondear a 2 decimales
        df['tradevalue'] = df['tradevalue'].round(2)
        #eliminar registros con tradevalue < 0
        df = df[df['tradevalue'] > 0]
        print(df.shape)
        return [df]

    def get_table(self,query):
        cur = self.conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        table = pd.DataFrame(rows,columns=[x[0] for x in cur.description])
        cur.close()
        return table 
        
    def obtainCountryList(self):
        df = self.read_data()[0]
        country_list= df['name'].unique()
        return country_list

    def grafica_treemap_paises(self,df,periodo=None,imp_exp=None):
        imp_exp = 'Importaciones' if imp_exp == 1 else 'Exportaciones'
        fig = px.treemap(df, 
                        path=['description','iso_3'], 
                        values='tradevalue',
                        title='{imp_exp} en millones de dólares ({año})'.format(imp_exp=imp_exp,año=periodo[0])
                        #color='porcentaje',
                        )
        fig.show()
    def grafica_treemap_paises_productos(self,df,periodo=None,imp_exp=None):
        imp_exp = 'Importaciones' if imp_exp == 1 else 'Exportaciones'
        fig = px.treemap(df, 
                        path=['description','SA_4'], 
                        values='tradevalue',
                        branchvalues='total',
                        #color='tradevalue',
                        title = '{imp_exp} en millones de dólares ({año})'.format(imp_exp=imp_exp,año=periodo[0]),
                        )
        fig.show()
    def grafica_destino_origen(self,df,periodo=None,imp_exp=None):
        imp_exp = 'Orígenes' if imp_exp == 1 else 'Destinos'
        df = df.groupby(['region','name','iso_3'],as_index=False)['tradevalue'].sum()
        total = df['tradevalue'].sum()
        df['porcentaje'] = df.groupby(['name'],group_keys=False)['tradevalue'].apply(lambda x: (x/total)*100)
        # #eliminar registros con porcentaje < 3
        # df = df[df['porcentaje'] > 3]
        # #df.tradevalue.quantile(0.25), df.tradevalue.quantile(0.7)
        # fig = px.choropleth(df, locations='iso_3', color='porcentaje',
        #                         color_continuous_scale="Viridis",
        #                         range_color=(df.tradevalue.quantile(0.25), df.tradevalue.quantile(0.9)), # range of values
        #                         labels={'aumento_disminucion':'porcentaje'},
        #                         title='{imp_exp} ({año})'.format(imp_exp=imp_exp,año=periodo[0])
        #                   )
        # fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})                                
        # fig.show()
        fig = px.treemap(df, 
                        path=['region','iso_3'], 
                        values='tradevalue',
                        branchvalues='total',
                        #color='tradevalue',
                        title = '{imp_exp} ({año})'.format(imp_exp=imp_exp,año=periodo[0]),
                        )    
        fig.show()
    def cambio_porcentualImpExp(self):
        df = self.read_data()[0]
        df =df.groupby(['year','imp_exp','partner_code','iso_3'],group_keys=False)[['tradevalue']].sum().reset_index()
        #paises unicos
        paises = df['partner_code'].unique()
        aumento_disminucion_pais = {}
        for pais in paises:
            df_pais = df[df['partner_code']==pais]
            df_pais=df_pais['tradevalue'].values.tolist()
            if len(df_pais) > 1:
                diff = ((df_pais[1]-df_pais[0])/df_pais[1]*100)
                aumento_disminucion_pais[pais] = diff
                #dicc to dataframe
        df_ = pd.DataFrame.from_dict(aumento_disminucion_pais,orient='index',columns=['aumento_disminucion'])
        df_['partner_code'] = df_.index
        df = df.merge(df_,how='inner',on='partner_code')
        return df
    
    def grafica_incrementoMercado_pais(self,df,scope_=None,periodo=None,imp_exp=None):
        periodo.sort()
        imp_exp = 'Importaciones' if imp_exp == 1 else 'Exportaciones'
        fig = px.choropleth(df, locations='iso_3', color='aumento_disminucion',
                                color_continuous_scale="Viridis",
                                range_color=(df.aumento_disminucion.quantile(0.25), df.aumento_disminucion.quantile(1)), # range of values
                                scope = scope_,
                                labels={'aumento_disminucion':'Aumento/disminucion'},
                                title='Cambio en el valor de importaciones del {} al {}'.format(periodo[0],periodo[1])
                          )
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})                                
        fig.show()
    def ITrade_pricnipalesSocios(self):
        df = self.read_data()[0]

        df_usa = df[df['iso_3']=='USA']
        df_canada = df[df['iso_3']=='CAN']
        df_jpn= df[df['iso_3']=='JPN']
        df_ue = df[df['iso_3'].isin(['AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP','SWE','GBR'])]
        df_aliazna_pacifico = df[df['iso_3'].isin(['CHL','PER','COL'])]
        df_centroamerica = df[df['iso_3'].isin(['CUB','PAN','GTM','CRI','SLV','NIC','HND','DOM'])]

        #UNIR DATAFRAMES
        df = pd.concat([df_usa,df_canada,df_jpn,df_ue,df_aliazna_pacifico,df_centroamerica])
        #CREAR COLUMNA REGION
        df['region_'] = np.where(df['iso_3'].isin(['USA']),'USA',
                        np.where(df['iso_3'].isin(['CAN']),'CAN',
                        np.where(df['iso_3'].isin(['JPN']),'JPN',
                        np.where(df['iso_3'].isin(['AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP','SWE','GBR']),'UE',
                        np.where(df['iso_3'].isin(['CHL','PER','COL']),'ALIANZA PACIFICO',
                        np.where(df['iso_3'].isin(['CUB','PAN','GTM','CRI','SLV','NIC','HND','DOM']),'CENTROAMERICA',''))))))

        #IMPORTACIONES
        df_imp = df[(df['imp_exp']==1)]
        df.groupby(['year','imp_exp','region_'],group_keys=False)[['tradevalue']].sum().reset_index()
        df_imp = df_imp.groupby(['year','imp_exp','region_'],group_keys=False)[['tradevalue']].sum().reset_index()
        df_imp = df_imp.pivot(index='year',columns='region_',values='tradevalue')

        #EXPORTACIONES
        df_exp = df[(df['imp_exp']==2)]
        df_exp.groupby(['year','imp_exp','region_'],group_keys=False)[['tradevalue']].sum().reset_index()
        df_exp = df_exp.groupby(['year','imp_exp','region_'],group_keys=False)[['tradevalue']].sum().reset_index()
        df_exp = df_exp.pivot(index='year',columns='region_',values='tradevalue')
        fig = make_subplots(rows=2, cols=3,subplot_titles=('USA','CAN','UE','JPN','ALIANZA PACIFICO','CENTROAMERICA'))
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['USA'],line_color='green',name='Importaciones'),row=1, col=1),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['USA'],line_color='blue',name='Exportaciones'),row=1, col=1),
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['CAN'],line_color='green',name='Importaciones',showlegend=False),row=1, col=2),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['CAN'],line_color='blue',name='Exportaciones',showlegend=False),row=1, col=2),
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['UE'],line_color='green',name='Importaciones',showlegend=False),row=1, col=3),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['UE'],line_color='blue',name='Exportaciones',showlegend=False),row=1, col=3),
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['JPN'],line_color='green',name='Importaciones',showlegend=False),row=2, col=1),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['JPN'],line_color='blue',name='Exportaciones',showlegend=False),row=2, col=1),
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['ALIANZA PACIFICO'],line_color='green',name='Importaciones',showlegend=False),row=2, col=2),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['ALIANZA PACIFICO'],line_color='blue',name='Exportaciones',showlegend=False),row=2, col=2),
        fig.add_trace(go.Scatter(x=df_imp.index, y=df_imp['CENTROAMERICA'],line_color='green',name='Importaciones',showlegend=False),row=2, col=3),
        fig.add_trace(go.Scatter(x=df_exp.index, y=df_exp['CENTROAMERICA'],line_color='blue',name='Exportaciones',showlegend=False),row=2, col=3),
        fig.update_layout(height=600, width=800, title_text="Intercambio comercial con socios princiapales (2015-2021)")
        fig.show()


c:\Users\ivan_\OneDrive - UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO\Desktop\repositorios\suayedApp\venv_\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
c = Data('world_trade',reporting_country='484',year=[2020],section=1)
df = c.read_data()
#[['year','SA_4','tradevalue','porcentaje','description','iso_3']]
df = df[0].head(15)
df[['year','region','name','SA_4','imp_exp','tradevalue','porcentaje','description']]

(141, 12)


,year,region,name,SA_4,imp_exp,tradevalue,porcentaje,description
0,2020,Europe,Germany,0101,1,21.97,35.57,Animales vivos y productos del reino animal
1,2020,Europe,Belgium,0101,1,4.02,6.51,Animales vivos y productos del reino animal
2,2020,America,United States of America,0101,1,32.32,52.33,Animales vivos y productos del reino animal
3,2020,America,United States of America,0102,1,43.91,97.63,Animales vivos y productos del reino animal
4,2020,America,United States of America,0102,2,883.17,100.00,Animales vivos y productos del reino animal
5,2020,America,Canada,0105,1,4.08,14.48,Animales vivos y productos del reino animal
6,2020,Europe,France,0105,1,0.96,3.41,Animales vivos y productos del reino animal
7,2020,America,United States of America,0105,1,23.16,82.11,Animales vivos y productos del reino animal
8,2020,Europe,Belgium,0106,1,0.31,9.36,Animales vivos y productos del reino animal
9,2020,Europe,Spain,0106,1,0.71,21.81,Animales vivos y productos del reino animal


## ¿Qué productos se exportaron más el año 2020?

In [11]:
c = Data('world_trade',reporting_country='484',year=[2019],imp_exp=2)
df = c.read_data()
c.grafica_treemap_paises_productos(df[0],periodo=c.year,imp_exp=c.imp_exp)

(2385, 12)


## ¿A dónde se exportaron esos productos en el 2020?

In [12]:
c = Data('world_trade_',reporting_country='484',year=[2020],imp_exp=2)
df = c.read_data()
c.grafica_destino_origen(df[0],periodo=c.year,imp_exp=c.imp_exp)

(2325, 12)


## ¿Qué productos se importaron más el año 2020?

In [13]:
c = Data('world_trade_',reporting_country='484',year=[2020],imp_exp=1,partner_code= '842')
df = c.read_data()
c.grafica_treemap_paises_productos(df[0],periodo=c.year,imp_exp=c.imp_exp)

(1089, 12)


## ¿De dónde se importaron esos productos ?

In [14]:
c = Data('world_trade_',reporting_country='484',year=[2020],imp_exp=1)
df = c.read_data()
c.grafica_destino_origen(df[0],periodo=c.year,imp_exp=c.imp_exp)

(4427, 12)


## ¿Cómo incrementó o disminuyó el comercio exterior con otros países?

In [20]:
c = Data('world_trade_',reporting_country='484',year=[2018,2019],imp_exp=1)
m = c.cambio_porcentualImpExp()
c.grafica_incrementoMercado_pais(m,scope_='world',periodo=c.year,imp_exp=c.imp_exp)

## Relaciones comerciales con socios principales

In [3]:
c = Data('world_trade',reporting_country='484',year=[i for i in range(2015,2022)])
c.ITrade_pricnipalesSocios()

(47503, 12)


In [ ]:
country_list = Data('world_trade_',reporting_country='484')
country_list = country_list.obtainCountryList()
country_list



In [ ]:
c = Data('world_trade_',reporting_country='484',year=[2020],imp_exp=2)
df = c.read_data()
c.grafica_treemap_paises(df[0],c.year,imp_exp=c.imp_exp)

In [8]:
with open(r'C:\Users\ivan_\OneDrive - UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO\Desktop\repositorios\Comercio_exterior\files/urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url.strip() for url in urls]
urls
if 'https://comtradeapi.un.org/public/v1/preview/C/A/HS?reporterCode=484&period=2021,2020&cmdCode=8536&flowCode=M,X&customsCode=C00&motCode=0' in urls:
    print('yes')
    

yes


In [1]:
import mysql.connector
import pandas as pd


class MysqlDB():
    def __init__(self):
       self.conn = mysql.connector.connect(user="root", password="123456",
                                       host="localhost",
                                       database="mexico_it",
                                       port='3306'
                                       )
    def insertar_registro(self,tabla,diccionario):
        cur = self.conn.cursor()
        if tabla =='world_trade_':
            insertar_registro = "INSERT INTO world_trade_ (reporter_country,year,section,SA_4, imp_exp,partner_code,tradevalue,fobvalue,tradequantity,quantity_unit,netweight) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cur.execute(insertar_registro, (diccionario['reporter_country'],
                                            diccionario['year'],
                                            diccionario['section'],
                                            diccionario['SA_4'],
                                            diccionario['imp_exp'],
                                            diccionario['partner_code'],
                                            diccionario['tradevalue'],
                                            diccionario['fobvalue'],
                                            diccionario['tradequantity'],
                                            diccionario['quantity_unit'],
                                            diccionario['netweight'],
                                            ))
        else:
            
            insertar_registro =  "INSERT INTO world_trade (reporter_country,year,section,SA_4, imp_exp,partner_code,tradevalue,fobvalue,tradequantity,quantity_unit,netweight) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            
            cur.execute(insertar_registro, (diccionario['reporter_country'],
                                            diccionario['year'],
                                            diccionario['section'],
                                            diccionario['SA_4'],
                                            diccionario['imp_exp'],
                                            diccionario['partner_code'],
                                            diccionario['tradevalue'],
                                            diccionario['fobvalue'],
                                            diccionario['tradequantity'],
                                            diccionario['quantity_unit'],
                                            diccionario['netweight'],
                                            ))
        self.conn.commit()
        #cur.close()
        #self.conn.close

    def crear_tabla_secciones(self):
        cur = self.conn.cursor()
        cur.execute("CREATE TABLE if not EXISTS sections (id INT AUTO_INCREMENT PRIMARY KEY, chapters VARCHAR(20))")
        cur.execute('''
                        INSERT INTO sections (chapters) VALUES
                        ('1-5'),
                        ('6-14'),
                        ('15'),
                        ('16-24'),
                        ('25-27'),
                        ('28-38'),
                        ('39-40'),
                        ('41-43'),
                        ('44-46'),
                        ('47-49'),
                        ('50-63'),
                        ('64-67'),
                        ('68-70'),
                        ('71'),
                        ('72-83'),
                        ('84-85'),
                        ('86-89'),
                        ('90-92'),
                        ('93'),
                        ('94-96'),
                        ('97')
                        ''')
        self.conn.commit()
        cur.close()
        self.conn.close()

    def consultar_sections(self):
        cur = self.conn.cursor()
        cur.execute("SELECT chapters FROM sections")
        rows = cur.fetchall()
        rows,= list(zip(*rows))
        self.conn.commit()
        cur.close()
        return rows
    
    def verificar_partner_code_in_countriesTable(self):
        cur = self.conn.cursor()
        cur.execute("SELECT distinct(partner_code)from world_trade_;")
        rows = cur.fetchall()
        rows,= list(zip(*rows))
        self.conn.commit()
        
        for row in rows:
            cur.execute("SELECT name FROM countries WHERE partner_code_ = %s", (row,))
            name = cur.fetchone()
            if name is None:
                print(row)
                #cur.execute("INSERT INTO countries (partner_code_) VALUES (%s)", (row,))
        cur.close()  
        return rows
    
    def eliminar_registros_insignificantes(self):
        cur = self.conn.cursor()
        cur.execute("SELECT * FROM world_trade_")
        rows = cur.fetchall()
        world_tradeTable = pd.DataFrame(rows,columns=[x[0] for x in cur.description])
        world_tradeTable = world_tradeTable[~(world_tradeTable['partner_code']=='0')] 
        world_tradeTable['porcentaje'] = world_tradeTable.groupby(['year','imp_exp','SA_4'],group_keys=False)['tradevalue'].apply(lambda x: (x/(x.sum()))*100)
        

        #crear una tabla con los mismo campos en la base de datos

        world_tradeTable_insignificantes = world_tradeTable[world_tradeTable['porcentaje']>1]
    
        #import dataframe into myqsl
        #world_tradeTable_insignificantes.to_sql('world_tradeTable_insignificantes', con=self.conn, if_exists='replace', index=False)
        print(world_tradeTable_insignificantes.shape)
        #añadir world a una nueva tabla en mysql
        #crear otro df a partir del registro 10000 de world_tradeTable_insignificantes

        world_tradeTable_insignificantes=world_tradeTable_insignificantes.iloc[113949:,:]
        print(world_tradeTable_insignificantes.shape)
        c = 0
        for row in world_tradeTable_insignificantes.itertuples():
            
            data = {}
            data['reporter_country'] = row.reporter_country
            data['year'] = row.year
            data['imp_exp'] = row.imp_exp
            data['partner_code'] = row.partner_code
            data['tradevalue'] = row.tradevalue
            data['tradequantity'] = row.tradequantity
            data['quantity_unit'] = row.quantity_unit
            data['netweight'] = row.netweight
            data['fobvalue'] = row.fobvalue
            data['section'] = row.section
            data['SA_4'] = row.SA_4
            if c%100==0:
                print(c)
            self.insertar_registro('world_trade',data)
            c+=1
        
        #comit
        self.conn.commit()
        cur.close()
        self.conn.close()


        

c = MysqlDB().eliminar_registros_insignificantes()


c:\Users\ivan_\OneDrive - UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO\Desktop\repositorios\suayedApp\venv_\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


(136923, 12)
(22974, 12)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
1